# Predicting New York City Cab Fare

This files in this notebook were downloaded from the Kaggle Competition "New York City Taxi Fare Prediction" at https://www.kaggle.com/c/new-york-city-taxi-fare-prediction.

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Data Wrangling

In [15]:
%%time
df = pd.read_csv('train.csv')

CPU times: user 2min 1s, sys: 1min 11s, total: 3min 12s
Wall time: 3min 40s


In [ ]:
import feather

In [16]:
%%time
# Save into feather format, about 1.5Gb. 
df.to_feather('nyc_cab_fare_raw.feather')

SyntaxError: invalid syntax (<ipython-input-16-9a1ff9689883>, line 2)

In [ ]:
%%time
# load the same dataframe next time directly, without reading the csv file again!
df = pd.read_feather('nyc_cab_fare_raw.feather')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

The max and min values seems like errors. More on that later. The dropoffs have a count less than the pickups. We can check for NaNs. 

### NaN Values

In [ ]:
# Count number of reviews with no dropoff latitude
df.dropoff_latitude.isnull().sum()

In [ ]:
# Check for null values
dfn = df[df.isnull().any(axis=1)]

In [ ]:
# Display null values
dfn.head(10)

Since these rows all include fare amounts, they should be eliminated from the data. Had the cab just been running idle, the dropoff and pickup would have the same coordinates. Also note the pickup coordinates of 0. This must also be an error.

In [ ]:
# Eliminate null values
df = df.dropna(axis=0, subset=['dropoff_latitude'])

In [ ]:
# Verify that counts are the same
df.describe()

### Latitude Longitude Corrections

According to https://www.mapdevelopers.com/geocode_bounding_box.php, New York City has the following bounds: 
North Latitude: 40.917577 
South Latitude: 40.477399 
East Longitude: -73.700272 
West Longitude: -74.259090

In [ ]:
# Find cab rides whose pickup or dropoff are within NYC boundaries
# Declare constants
latmin = 40.477399
latmax = 40.917577
longmin = -74.259090
longmax = -73.700272

# Create dataframe with correct coordinates
df = df[((((df['pickup_longitude']<=longmax) & (df['pickup_longitude']>=longmin)) & ((df['pickup_latitude']<=latmax) & (df['pickup_latitude']>=latmin)))) | ((((df['dropoff_longitude']<=longmax) % (df['dropoff_longitude']>=longmin)) & ((df['dropoff_latitude']<=latmax) & (df['dropoff_latitude']>=latmin))))]

In [ ]:
df.describe()

Some of the entries still don't make sense. Whereas the pickup might be correct, the min/max reveal dropoff

According to https://www.mapdevelopers.com/geocode_bounding_box.php, the boundary of the United States is as follows.
North Latitude: 71.538800 
South Latitude: 18.776300 
East Longitude: -66.885417 
West Longitude: 170.595700

Sticking to the Mainland, California gives the following:
West Longitude: -124.482003

In [ ]:
# Choose cab rides whose pickup and dropoff are the US Mainland
# Declare constants
latmin = 5.496100
latmax = 71.538800
longmin = -124.482003
longmax = -66.885417

# Create dataframe with correct coordinates
df = df[((((df['pickup_longitude']<=longmax) & (df['pickup_longitude']>=longmin)) & ((df['pickup_latitude']<=latmax) & (df['pickup_latitude']>=latmin)))) & ((((df['dropoff_longitude']<=longmax) & (df['dropoff_longitude']>=longmin)) & ((df['dropoff_latitude']<=latmax) & (df['dropoff_latitude']>=latmin))))]

In [ ]:
df.describe()

### Fare / Passenger Count Corrections

There still appear to be anomalies. There should not be 208 passengers in a taxi, or 0. It is conceivable that there could be 0 passengers, and money could be lost, but since the data includes pickups and dropoffs, and the goal is to predict cab fare, the entries with 0 passengers can be eliminated. Furthermore, this 208 passenger anomaly could be a cross state bus ride. It can also be eliminated. According to https://www1.nyc.gov/nyc-resources/faq/484/how-many-passengers-are-allowed-in-a-taxiIf, 4 or 5 people are allowed in a taxi, plus one child under 9. I'll assume a couple more can crowd in and the driver will allow it.

In [ ]:
# Only choose cabs between 1 and 9 riders
df = df[(df['passenger_count'] < 10) & (df['passenger_count'] > 0)]

In [ ]:
df.describe()

This looks better, but there are still problems with the min and the max. Also, that max fare keeps coming up with a different passenger_count.

In [ ]:
df[df['fare_amount']>=10000]

It is possible that people are going to California and back, but even so, this messes with the data. We are presuming that the dropoff is the goal destination, but this won't always be the case.

In [ ]:
# Choose only cab rides that are less than 10,000.
df = df[df['fare_amount']<10000]

No one should have a negative cab ride. According to Dr. Google, the NYC Taxi base fare recently increased from 2 to 2.50. It's unclear what the minimum cab fare should be for this dataset. A histogram should verify things.

In [ ]:
# Setup Seaborn
import seaborn as sns
sns.set_style("darkgrid")

# Create histogram of 20 bins showing fares from 0 to $20
plt.hist(df.fare_amount, bins=21, range=(0,20), ec='white')

# Create labels
plt.xlabel('$ Cab Fare')
plt.ylabel('Passenger Count')
plt.title('Cab Fares Under $20 ', fontsize=15)

# Save figure
plt.savefig("Cab_Fares_Under_20_Hist", dpi=300)

# Show graph
plt.show()

The histogram reveals that the mininum cab fare was $2 during the time period that the data was colleged

In [ ]:
# Eliminate cab rides less than $2
df = df[df['fare_amount'] >= 2]

In [ ]:
df.describe()

Still a couple of issues. The min fare looks good, but the max seems quite high. Also, the dropoff latitude and longitude of the min are way out of range.

One idea is to create a new column, distance, and then plot it against fare to determine outliers. While it's true that someone can rent a cab for hours zigzagging across NYC, and return to where they started, this kind of fare is going to distort the data.

In [ ]:

plt.hist(df.fare_amount, bins=20, ec='white', log=True)
plt.xlabel('$ Cab Fare')
plt.ylabel('Passenger Count')
plt.title('Log Cab Fare Histogram', fontsize=15)
plt.savefig("Cab_Fare_Hist", dpi=300)
plt.show()

In [ ]:
# Save file before adding new column
df.to_csv('NYC_Cab_Fare_Wrangled.csv')

### Euclidean and Taxicab Distance

I am going to define both a Euclidean Distance, an unrealistic min since the shortest distance between 2 points is not a straight line through buildings and blocks, and a Taxicab Distance, a more realistic expectation of miles traveled. The Euclidean distance will serve as a min to eliminate outliers and errors. According to http://nymag.com/nymetro/urban/features/taxi/n_20286/, one could not do bretter than $1.50 per mile. 

In [ ]:
# Define coordinates (x,y)
x1 = df['pickup_latitude']
y1 = df['pickup_longitude']
x2 = df['dropoff_latitude']
y2 = df['dropoff_longitude']

# Create Euclidean Distrance column
df['euclidean_distance'] = np.sqrt((y2-y1)**2 + (x2-x1)**2)

# Create Taxicab Distance column
df['taxicab_distance'] = np.abs(y2-y1) + np.abs(x2-x1)

# Convert to miles
df['euclidean_distance'] = df['euclidean_distance'] * 69
df['taxicab_distance'] = df['taxicab_distance'] * 69

# Show first five entries
df.head()

#### Distance / Fare Plots

In [ ]:
# Define x and y
x = df.euclidean_distance
y = df.fare_amount

# Create scatter plot
plt.scatter(x, y, alpha=0.2)

# Create labels
plt.ylabel('Fare')
plt.xlabel('Miles')

# Create title
plt.title('NYC Cab Cost per Mile', fontsize=15)

#Since helpful votes cannot exceed total votes, the cyan line represents a maximum. 
#All dots must be underneath the line
minLine, = plt.plot(x, 1.5*x, 'plum', label='Min_cost')

plt.legend(handles=[minLine])
plt.savefig('Cab Cost per Mile', dpi=300)
plt.show()

All rides underneath the plum line can be eliminated. Since the going rate was 2.00 per cab at 1.50 per mile in NYC, they are impossible.

In [ ]:
# Define x and y
x = df.euclidean_distance
y = df.fare_amount

# Create scatter plot
plt.scatter(x, y, alpha=0.2)

# Create labels
plt.ylabel('Fare')
plt.xlabel('Miles')

# Create zoom limits
plt.xlim(0, 25)
plt.ylim(0, 25)

# Create title
plt.title('NYC Cab Cost per Mile', fontsize=15)

#Since helpful votes cannot exceed total votes, the cyan line represents a maximum. 
#All dots must be underneath the line
minLine, = plt.plot(x, 1.5*x, 'plum', label='Min_cost')

plt.legend(handles=[minLine])
plt.savefig('Cab Cost per Mile', dpi=300)
plt.show()

In [ ]:
#### Eliminate unrealistic plots
df = df[df['fare_amount'] >= (df['euclidean_distance'] * 1.5 + 2)]

In [ ]:
# Elminate fares that traveled no distance
df = df[df['euclidean_distance']>0]

In [ ]:
plt.hist(df.fare_amount, bins=20, ec='white', log=True)
plt.xlabel('$ Cab Fare')
plt.ylabel('Passenger Count')
plt.title('Log Cab Fare Histogram', fontsize=15)
plt.savefig("Cab_Fare_Hist", dpi=300)
plt.show()

### Save File

In [ ]:
df.to_csv('NYC_Cab_Fare_Wrangled.csv')

## Time Series

In [ ]:
# df = pd.read_csv('NYC_Cab_Fare_Wrangled.csv')

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
type(df.pickup_datetime[0])

In [ ]:
def cut_UTC(row):
    row = row.split(' UTC')
    return row[0]

In [ ]:
df['pickup_datetime'] = df['pickup_datetime'].apply(cut_UTC)

In [ ]:
df['pickup_datetime']=pd.to_datetime(df['pickup_datetime'], errors='ignore')

In [ ]:
df.info()

In [ ]:
df['month'] = df['pickup_datetime'].dt.month
df['year'] = df['pickup_datetime'].dt.year
df['hour'] = df['pickup_datetime'].dt.hour
df['minute'] = df['pickup_datetime'].dt.minute
df['second'] = df['pickup_datetime'].dt.second
df['dayofweek'] = df['pickup_datetime'].dt.dayofweek

In [ ]:
df['15_min_intervals'] = 4 * df['hour'] + (df['minute']/15).astype(int)
df['total_seconds'] = 3600 * df['hour'] + 60 * df['minute'] + df['second']

In [ ]:
def summer_month(row):
    if row['month'] in [6,7,8]:
        return 1
    else:
        return 0

df['summer_month'] = df.apply(summer_month, axis=1)

In [ ]:
def cold_month(row):
    if row['month'] in [1,2,3,11,12]:
        return 1
    else:
        return 0
    
df['cold_month'] = df.apply(cold_month, axis=1)

In [ ]:
def weekend(row):
    if row['dayofweek'] in [5,6]:
        return 1
    else:
        return 0

df['weekend'] = df.apply(weekend, axis=1)

In [ ]:
def rush_hour(row):
    if (row['hour'] in [7,8,9,15,16,17,18,19]) & (row['weekend'] == 0):
        return 1
    else:
        return 0

df['rush_hour'] = df.apply(rush_hour, axis=1)

In [ ]:
def night_rush(row):
    if (row['hour'] in [19,20,21,22,23,24,1]) & (row['dayofweek'] in [3,4,5]):
        return 1
    else:
        return 0
    
df['night_rush'] = df.apply(night_rush, axis=1)

In [ ]:
def night_charge(row):
    if row['hour'] in [20,21,22,23,24,1,2,3,4,5,6]:
        return 1
    else:
        return 0
    
df['night_charge'] = df.apply(night_charge, axis=1)

In [ ]:
def weekday_surcharge(row):
    if (row['hour'] in [16,17,18,19,20]) & (row['dayofweek'] in [1,2,3,4,5]):
        return 1
    else:
        return 0
    
df['weekday_surcharge'] = df.apply(weekday_surcharge, axis=1)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.to_csv('NYC_Cab_Fare_Wrangled_Time_Distance.csv')

## Holiday Additions

In [ ]:
#from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

#dr = pd.date_range(start='2015-07-01', end='2015-07-31')
#df = pd.DataFrame()
#df['Date'] = dr

#cal = calendar()
#holidays = cal.holidays(start=dr.min(), end=dr.max())

#df['Holiday'] = df['Date'].isin(holidays)
#print df

## Latitude Longitude Additions

In [ ]:
df = pd.read_csv('NYC_Cab_Fare_Wrangled_Time_Distance.csv')
df.head()

I used https://getlatlong.net/ to estimate a geographical area for Manhattan.

These are the 4 points, that when connected as a quadrilateral, enclose most of Manhattan. (I chose some points in the water since they are unaffected traffic

Then for any given point (x,y) both of the following conditions must hold:
bottom line <= (x,y) <= top line
right line <= (x,y) <= left line

Or, equilavlently,
bottom line <= mx + b <= top line
right line <= mx + b <= left line

For instance, to determine if (3,4) <= 2x+1,
the condition must be checked that
4 <= 2(3) + 1
which in this case is true.

In [ ]:
# Define line from two points and a provided column
def two_points_line(a, b, column):
        
    # Case when y-values are the same
    if b[1]==a[1]:
        
        # Slope defaults to 0
        slope = 0
        
    # Case when x-values are the same
    elif b[0]==a[0]:
        
        # Case when max value is less than 999999999
        if column.max() < 999999999:
            
            # Add 999999999 to max value
            slope = column.max() + 999999999
        
        # All other cases
        else:
            
            # Multiply max value by itself (greater than 999999999)
            slope = column.max() * column.max()
    
    # When x-values and y-values are not 0
    else:
        
        # Use standard slope formula
        slope = (b[1] - a[1])/(b[0]-a[0])
    
    
    # Equation for y-intercept (solving y=mx+b for b)
    y_int = a[1] - slope * a[0]
    
    # Return slope and y-intercept
    return slope, y_int

In [ ]:
upper_right = (-73.929224, 40.804328)
bottom_right = (-73.980036, 40.710706)
bottom_left = (-74.054880, 40.681292)
upper_left = (-73.966303, 40.830050)

In [ ]:
m_top, b_top = two_points_line(upper_right, upper_left, df.pickup_latitude)
m_left, b_left = two_points_line(bottom_left, upper_left, df.pickup_latitude)
m_right, b_right = two_points_line(bottom_right, upper_right, df.pickup_latitude)
m_bottom, b_bottom = two_points_line(bottom_right, bottom_left, df.pickup_latitude)

In [ ]:
def manhattan_test((a,b)(x,y)):
    if (((b >= (a * m_top + b_top)) &
    (b <= (a * m_bottom + b_bottom))) &
    ((b <= (a * m_right + b_right)) &
    (b >= (a * m_left + b_left)))) & (((y >= (x * m_top + b_top)) &
    (y <= (x * m_bottom + b_bottom))) &
    ((y <= (x * m_right + b_right)) &
    (y >= (x * m_left + b_left)))):
        return 1
    else:
        return 0

In [ ]:
print(manhattan_test((-73.947763, 40.805107), (-73.977289, 40.764035))) #expect 1
print(manhattan_test((-73.947763, 40.805107), (-73.918924, 40.754673))) #expect 0
print(manhattan_test((-73.919611, 40.710966), (-73.918924, 40.754673))) #expect 0

In [ ]:
def manhattan(row):
    if (((row['pickup_latitude'] <= (row['pickup_longitude'] * m_top + b_top)) &
    (row['pickup_latitude'] >= (row['pickup_longitude'] * m_bottom + b_bottom))) &
    ((row['pickup_latitude'] >= (row['pickup_longitude'] * m_right + b_right)) &
    (row['pickup_latitude'] <= (row['pickup_longitude'] * m_left + b_left)))) & (((row['dropoff_latitude'] <= (row['dropoff_longitude'] * m_top + b_top)) &
    (row['dropoff_latitude'] >= (row['dropoff_longitude'] * m_bottom + b_bottom))) &
    ((row['dropoff_latitude'] >= (row['dropoff_longitude'] * m_right + b_right)) &
    (row['dropoff_latitude'] <= (row['dropoff_longitude'] * m_left + b_left)))):
        return 1
    else:
        return 0

In [ ]:
df['manhattan'] = df.apply(manhattan, axis=1)

In [ ]:
def manhattan_to_fro(row):
    if ((((row['pickup_latitude'] <= (row['pickup_longitude'] * m_top + b_top)) &
    (row['pickup_latitude'] >= (row['pickup_longitude'] * m_bottom + b_bottom))) &
    ((row['pickup_latitude'] >= (row['pickup_longitude'] * m_right + b_right)) &
    (row['pickup_latitude'] <= (row['pickup_longitude'] * m_left + b_left)))) | (((row['dropoff_latitude'] <= (row['dropoff_longitude'] * m_top + b_top)) &
    (row['dropoff_latitude'] >= (row['dropoff_longitude'] * m_bottom + b_bottom))) &
    ((row['dropoff_latitude'] >= (row['dropoff_longitude'] * m_right + b_right)) &
    (row['dropoff_latitude'] <= (row['dropoff_longitude'] * m_left + b_left))))) &
        
    (((((row['pickup_latitude'] <= (row['pickup_longitude'] * m_top + b_top)) &
    (row['pickup_latitude'] >= (row['pickup_longitude'] * m_bottom + b_bottom))) &
    ((row['pickup_latitude'] >= (row['pickup_longitude'] * m_right + b_right)) &
    (row['pickup_latitude'] <= (row['pickup_longitude'] * m_left + b_left))))==False) | (((row['dropoff_latitude'] <= (row['dropoff_longitude'] * m_top + b_top)) &
    (row['dropoff_latitude'] >= (row['dropoff_longitude'] * m_bottom + b_bottom))) &
    ((row['dropoff_latitude'] >= (row['dropoff_longitude'] * m_right + b_right)) &
    (row['dropoff_latitude'] <= (row['dropoff_longitude'] * m_left + b_left)))==False)):
        return 1
    else:
        return 0

In [ ]:
df['manhattan_to_fro'] = df.apply(manhattan, axis=1)

## Save file

In [ ]:
df.to_csv('NYC_Cab_Fare_Wrangled_Time_Distance.csv')

## Distance Additions

In [ ]:
plt.hist(df['euclidean_distance'], bins=20, range=(0,1))
plt.title('Rides Less Than 1 Mile', fontsize=15)
plt.savefig('Cab Cost per Mile', dpi=300)
plt.show()

In [ ]:
plt.hist(df['euclidean_feet'], bins=20, range=(0,400))
plt.title('Rides Less Than 400 Feet', fontsize=15)
plt.savefig('Rides Less Than 400 Feet', dpi=300)
plt.show()

In [ ]:
plt.hist(df['euclidean_feet'], bins=20, range=(0,10))
plt.title('Rides Less Than 10 Feet', fontsize=15)
plt.savefig('Rides Less Than 10 Feet', dpi=300)
plt.show()

In [ ]:
y = df.fare_amount
x = df.euclidean_feet
plt.scatter(x, y, alpha=0.1)
plt.ylabel('Fare')
plt.xlabel('Feet')
plt.ylim(0,400)
plt.xlim(0,400)
plt.title('Cost per Mile NYC Cabs', fontsize=15)

In [ ]:
y = df.fare_amount
x = df.euclidean_feet
plt.scatter(x, y, alpha=0.1)
plt.ylabel('Fare')
plt.xlabel('Feet')
plt.ylim(0,100)
plt.xlim(0,10)
plt.title('Cost per Mile NYC Cabs', fontsize=15)